In [1]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from modules import solver, presolver
from pyballistics import ozvb_lagrange, get_powder_names
from tqdm import tqdm

## Нахождение начальных точек
формируется таблица с начальными точками для каждого пороха

In [2]:
tabel_init_points = pd.read_csv("data/init_points.csv")
tabel_init_points.loc[tabel_init_points['success']==False]

,dbname,wq,ro,reason,success


## Решение обратной задачи

In [3]:
tabel_result = list()

In [ ]:
%%time

for i, row in tqdm(tabel_init_points.iterrows()):
    initial_dict = presolver.init_dict(row['dbname'], row['wq'], row['ro'])
    #print("name: ", row['dbname'], " wq: ", row['wq'], " ro ", row['ro'])
    solv = solver.Solver(initial_dict)
    solv.solution()
    tabel_result.append(solv.get_solution())   

0it [00:00, ?it/s]

In [6]:
result = copy.deepcopy(tabel_result)

In [29]:
table = list()
for row in result:
    if row == {None}:
        continue
    table.append(row)

In [30]:
table = pd.DataFrame(table)

In [31]:
#table.to_csv("data/table_result.csv", index=False)

## Обработка

In [32]:
table.head()

,name,succes,wq,ro,mass
0,ДРП,True,3.869251,1011.404983,2650.941081
1,КЗДП,True,3.869251,1011.404979,2650.945574
2,СФ 033,True,1.935400,306.411056,992.756855
3,ВТ,True,1.613720,341.222446,928.111264
4,ВТМ,True,1.044243,429.070729,1769.155477


In [38]:
table.sort_values(by=['mass'])

,name,succes,wq,ro,mass
81,УГ-2,True,0.866869,89.828520,300.690783
34,12/7 В/А,True,1.327616,885.092813,301.055455
36,12/7,True,1.547683,897.894231,305.972314
41,15/1 тр В/А,True,1.383710,866.740867,306.791717
40,ДГ-4 13/1,True,1.494576,785.134073,315.032846
...,...,...,...,...,...
0,ДРП,True,3.869251,1011.404983,2650.941081
1,КЗДП,True,3.869251,1011.404979,2650.945574
17,ВТХ-20,True,2.926365,498.787279,3182.752509
57,УГ-1,True,8.138917,1132.513282,3280.681214
